# Genome alignment


In [2]:
pip install pytrimal biopython dendropy

ERROR: Could not find a version that satisfies the requirement trimal (from versions: none)
ERROR: No matching distribution found for trimal


In [ ]:
!apt-get install -qq -y mafft

In [ ]:
!wget https://github.com/inab/trimal/archive/refs/tags/v1.4.1.tar.gz
!tar -xf v1.4.1.tar.gz
%cd trimal-1.4.1/source
!make

In [2]:
import os
import dendropy
from Bio.Align.Applications import MafftCommandline

In [5]:
mafft_cline = MafftCommandline(input='sample.fasta', ep=0.123, reorder=True, maxiterate=1000, localpair=True, thread = -1)
print(mafft_cline)
stdout, stderr = mafft_cline()
with open('align.fasta', 'w') as w:
    w.write(stdout)

mafft --localpair --maxiterate 10 --thread -1 --ep 0.123 --reorder sample.fasta


to trim the output:
http://trimal.cgenomics.org/getting_started_with_trimal_v1.2

In [4]:
!trimal -automated1 -in align.fasta -out trim.fasta -fasta


ERROR: Alignment not loaded: "align.fasta" Check the file's content.



## Protein alignment

In [2]:
from Bio.Align.Applications import MuscleCommandline

In [14]:
my_genes = ['NP', 'L', 'VP35', 'VP40']

muscle_exe = r"D:\backup\Git\us\ptDtSc\Bioinformatics_with_Python\Chapter_7_Phylogenetics\muscle.exe"

for gene in my_genes:
    muscle_cline = MuscleCommandline(cmd=muscle_exe, input='%s_P.fasta' % gene)
    print(muscle_cline)
    stdout, stderr = muscle_cline()
    with open('%s_P_align.fasta' % gene, 'w') as w:
        w.write(stdout)

D:\backup\Git\us\ptDtSc\Bioinformatics_with_Python\Chapter_7_Phylogenetics\muscle.exe -in NP_P.fasta
D:\backup\Git\us\ptDtSc\Bioinformatics_with_Python\Chapter_7_Phylogenetics\muscle.exe -in L_P.fasta
D:\backup\Git\us\ptDtSc\Bioinformatics_with_Python\Chapter_7_Phylogenetics\muscle.exe -in VP35_P.fasta
D:\backup\Git\us\ptDtSc\Bioinformatics_with_Python\Chapter_7_Phylogenetics\muscle.exe -in VP40_P.fasta


In [16]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
# XXX vvv
# from Bio.Alphabet import generic_protein

for gene in my_genes:
    gene_seqs = {}
    unal_gene = SeqIO.parse('%s.fasta' % gene, 'fasta')
    for rec in unal_gene:
        gene_seqs[rec.id] = rec.seq

    al_prot = SeqIO.parse('%s_P_align.fasta' % gene, 'fasta')
    al_genes = []
    for protein in al_prot:
        my_id = protein.id
        seq = ''
        pos = 0
        for c in protein.seq:
            if c == '-':
                seq += '---'
            else:
                seq += str(gene_seqs[my_id][pos:pos + 3])
                pos += 3
        al_genes.append(SeqRecord(Seq(seq), id=my_id))


    SeqIO.write(al_genes, '%s_align.fasta' % gene, 'fasta')